In [1]:
!yes | pip uninstall torchvison
!pip install -qU torchvision

yes: standard output: Broken pipe
yes: write error
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.5.1 requires torch<1.10,>=1.7.0, but you have torch 1.10.1 which is incompatible.


In [3]:
import os
import time
import boto3
import numpy as np
import pandas as pd
import sagemaker
from sagemaker import get_execution_role

In [64]:

sess = boto3.Session()
sm = sess.client("sagemaker")
sagemaker_session = sagemaker.Session(boto_session=sess)
role = get_execution_role()
bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/rain-au-pytorch-notebook'

print(f"Bucket : {bucket}/{prefix} \n Role : {role} \n Session: {sagemaker_session}")

Bucket : sagemaker-eu-west-3-543553163241/sagemaker/rain-au-pytorch-notebook 
 Role : arn:aws:iam::543553163241:role/service-role/AmazonSageMaker-ExecutionRole-20220116T155750 
 Session: <sagemaker.session.Session object at 0x7f987ff7c290>


In [46]:
region = boto3.Session().region_name
region

'eu-west-3'

In [9]:
BASE_DIR = "pipelines/rain/"
input_data_source = "s3://rain-data-17012022/data/weatherAUS.csv"

In [10]:
from sagemaker.workflow.parameters import ParameterInteger, ParameterString, ParameterFloat

base_job_prefix="rain-au"
model_package_group_name = "RainAuModel"
pipeline_name = "TrainingPipelineRainAu"  # SageMaker Pipeline name

# parameters for pipeline execution
processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount", default_value=1)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType", default_value="ml.m5.xlarge"
)
training_instance_type = ParameterString(
    name="TrainingInstanceType", default_value="ml.m5.xlarge"
)
inference_instance_type = ParameterString(
    name="InferenceInstanceType", default_value="ml.m5.xlarge"
)
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)

# Training
input_data = ParameterString(
    name="InputDataUrl",
    default_value=input_data_source    # default_value=f"s3://sagemaker-servicecatalog-seedcode-{region}/dataset/abalone-dataset.csv",
)
training_epochs = ParameterString(name="TrainingEpochs", default_value="1")

# Validation
# Low threshold as it is only an example
accuracy_threshold = ParameterFloat(name="AccuracyThreshold", default_value=0.6) 

# Getting and preprocessing the data
Data is stored in an s3 bucket in the account, needs to be processed by an SKlearn processor.

First let us update the git repository

In [17]:
!git pull origin main

From https://github.com/gmguarino/sagemaker-pipeline-rain-australia-build
 * branch            main       -> FETCH_HEAD
Already up to date.


In [20]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

Examine the preprocessing script

In [19]:
!pygmentize pipelines/rain/preprocess.py

import logging
import sys

import argparse
import os
import pathlib
import pickle
import joblib
import tarfile

import boto3
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))


def merge_two_dicts(x, y):
    """Merges two dicts, returning a new copy."""
    z = x.copy()
    z.update(y)
    return z


def cyclical_encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data


if __name__ == "__main__":

    logger.debug("Starting preprocessing.")
    # parser = argparse.ArgumentParser()
    # parser.add_argument

In [21]:
sklearn_framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version=sklearn_framework_version,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    volume_size_in_gb=15,
    base_job_name="rainau-notebook-pipeline-preprocess",
    role=role
)

In [54]:
processing_step = ProcessingStep(
    name="RainAuProcessing",
    processor=sklearn_processor,
    inputs=[ProcessingInput(source=input_data,
        destination='/opt/ml/processing/input/')],
    outputs=[
        ProcessingOutput(output_name="train",
            source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="validation",
            source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test",
            source="/opt/ml/processing/test"),
        ProcessingOutput(output_name="scaler_model",
            source="/opt/ml/processing/preprocess")
    ],
    code=os.path.join(BASE_DIR, "preprocess.py")
)

In [23]:
model_path = f"s3://{sagemaker_session.default_bucket()}/{base_job_prefix}/model"

In [24]:
model_path

's3://sagemaker-eu-west-3-543553163241/rain-au/model'

In [33]:
from sagemaker.pytorch import PyTorch

# Train the model in a Pytorch container

The script `train.py` takes care of the training and saving of the neural network

In [34]:
!git pull origin main

From https://github.com/gmguarino/sagemaker-pipeline-rain-australia-build
 * branch            main       -> FETCH_HEAD
Already up to date.


In [35]:
!pygmentize pipelines/rain/train.py

import argparse
import json
import logging
import os
import sys
import shutil
import tarfile
import pathlib

import pandas as pd
import numpy as np

#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import Dataset

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))


# Dataset class to handle data loading
class RainDataset(Dataset):
    def __init__(self, csv_file, root_dir):
        self.data = pd.read_csv(os.path.join(root_dir, csv_file))
        self.labels = np.asarray(self.data.iloc[:, 0])
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        single_label = self.labels[idx]

        data_array = np.asarray(self.data.i

In [87]:
pytorch_estimator = PyTorch(
    entry_point='train.py',
    source_dir=BASE_DIR,
    instance_type=training_instance_type,
    instance_count=1,
    framework_version='1.8.0',
    py_version='py3',
    base_job_name=f"{base_job_prefix}-torch-train-notebook-pipeline",
    output_path=model_path,
    hyperparameters={'epochs': training_epochs, 'batch-size': 32, 'lr': 0.00009},
    role=role
)

In [88]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

step_train = TrainingStep(
    name="RainAuPytorchModel",
    estimator=pytorch_estimator,
    inputs={
        "train": TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
            content_type="text/csv",
        )
    }
)

# Evaluate the model

In [43]:
!git pull origin main

From https://github.com/gmguarino/sagemaker-pipeline-rain-australia-build
 * branch            main       -> FETCH_HEAD
Already up to date.


In [44]:
from sagemaker.processing import ScriptProcessor, FrameworkProcessor
from sagemaker.workflow.properties import PropertyFile

In [79]:
!pygmentize pipelines/rain/evaluate.py

"""Evaluation script for measuring mean squared error."""
import json
import logging
import pathlib
import os
import argparse
import sys

import pandas as pd
import numpy as np

import tarfile

#import sagemaker_containers
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import Dataset

from sklearn.metrics import (accuracy_score, 
    precision_score, recall_score, f1_score)

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))

# Dataset class to handle data loading
class RainDataset(Dataset):
    def __init__(self, csv_file, root_dir):
        self.data = pd.read_csv(os.path.join(root_dir, csv_file))
        self.labels = np.asarray(self.data.iloc[:, 0])
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        single_la

In [52]:
#pytorch_processor = FrameworkProcessor(
#    PyTorch,
#    instance_type=processing_instance_type,
#    instance_count=1,
#    framework_version='1.8.0',
#    base_job_name=f"{base_job_prefix}-torch-eval",
#    sagemaker_session=sagemaker_session,
#    role=role,
#    command=["python3"],
#)

pytorch_eval_image = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=region,
    version='1.8.0',
    image_scope="training",
    py_version="py3",
    instance_type=training_instance_type
)

evaluation_processor = ScriptProcessor(
    role=role,
    image_uri=pytorch_eval_image,
    command=["python3"],
    instance_count=1,
    instance_type=training_instance_type
)

#References outputs from the processing step
evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)

step_evaluate = ProcessingStep(
    name="EvaluatePerformance",
    processor=evaluation_processor,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=processing_step.properties.ProcessingOutputConfig.Outputs["validation"].S3Output.S3Uri,
            destination="/opt/ml/processing/validation",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    property_files=[evaluation_report],
    code=os.path.join(BASE_DIR, "evaluate.py")
)

# Register the model on condition

In [55]:
scaler_model_artifacts = "{}/model.tar.gz".format(
    processing_step.arguments["ProcessingOutputConfig"]["Outputs"][3]["S3Output"]["S3Uri"]
)
scaler_model_artifacts

's3://sagemaker-eu-west-3-543553163241/RainAuProcessing-e1284f24b646de14ac1aa45ecdee541d/output/scaler_model/model.tar.gz'

## Load the models for registration

In [74]:
from sagemaker.sklearn.model import SKLearnModel
from sagemaker.pytorch.model import PyTorchModel
from sagemaker import PipelineModel

# Load the scaler sklearn model artifacts into a SKLearnModel class

scaler_model = SKLearnModel(
    model_data=scaler_model_artifacts,
    role=role,
    sagemaker_session=sagemaker_session,
    entry_point=os.path.join(BASE_DIR, "preprocess.py"), # The handler functions are defined here as well
    framework_version=sklearn_framework_version, # as before
)


# Load the pytorch model artifacts
pytorch_model = PyTorchModel(
    entry_point='inference.py',
    source_dir=BASE_DIR,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    role=role,
    framework_version='1.8.0',
    py_version='py3',
    name="rain-au-inference-model-pipeline",
    sagemaker_session=sagemaker_session # remember to always have this
)

# Combine them into a single model
pipeline_model = PipelineModel(
    models=[scaler_model, pytorch_model], role=role, sagemaker_session=sagemaker_session
)

In [75]:
sagemaker_session

In [76]:
# Registration step

from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.step_collections import RegisterModel


evaluation_s3_uri = "{}/evaluation.json".format(
    step_evaluate.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
)

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=evaluation_s3_uri,
        content_type="application/json",
    )
)

step_register_pipeline_model = RegisterModel(
    name="RainAuPipelineModel",
    model=pipeline_model,
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=["ml.m5.large", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    model_metrics=model_metrics,
    approval_status=model_approval_status,
)


# Create condition for registration

In [77]:
!git pull origin main

From https://github.com/gmguarino/sagemaker-pipeline-rain-australia-build
 * branch            main       -> FETCH_HEAD
Already up to date.


In [95]:
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

In [96]:

# basically this check that <left> <= <right>
condition = ConditionGreaterThanOrEqualTo(
    left=JsonGet( # Basically this reads the property file to find the necessary metric
        step_name=step_evaluate.name,
        property_file=evaluation_report,
        json_path="regression_metrics.accuracy", # probably should change path as it is not regression
    ),
    right=accuracy_threshold
)

# Turn this into a pipeline step
step_cond = ConditionStep(
    name="rain-au-accuracy-condition",
    conditions=[condition],
    if_steps=[step_register_pipeline_model],  # step_register_model, step_register_scaler,
    else_steps=[], # if the model does not pass then nothing happens
)


# Actually create the pipeline

In [97]:
from sagemaker.workflow.pipeline import Pipeline

# Create a Sagemaker Pipeline.
# Each parameter for the pipeline must be set as a parameter explicitly when the pipeline is created.
# Also pass in each of the steps created above.
# Note that the order of execution is determined from each step's dependencies on other steps,
# not on the order they are passed in below.
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[ # These are all the parameters defined at the beginning that can be passed and adapted
        # as the pipelines are executed
        processing_instance_type,
        processing_instance_count,
        training_instance_type,
        inference_instance_type,
        input_data,
        model_approval_status,
        training_epochs,
        accuracy_threshold,
    ],
    steps=[processing_step, step_train, step_evaluate, step_cond]
)

# View and inspect pipeline definition

In [99]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'ProcessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'InferenceInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'InputDataUrl',
   'Type': 'String',
   'DefaultValue': 's3://rain-data-17012022/data/weatherAUS.csv'},
  {'Name': 'ModelApprovalStatus',
   'Type': 'String',
   'DefaultValue': 'PendingManualApproval'},
  {'Name': 'TrainingEpochs', 'Type': 'String', 'DefaultValue': '1'},
  {'Name': 'AccuracyThreshold', 'Type': 'Float', 'DefaultValue': 0.6}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'RainAuProcessing',
   'Type': 'Processing',
   'Arguments': {'Process

# Submit pipeline to Sagemaker

In [100]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:eu-west-3:543553163241:pipeline/trainingpipelinerainau',
 'ResponseMetadata': {'RequestId': '437cc514-8e40-415a-9801-a2e042900c8e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '437cc514-8e40-415a-9801-a2e042900c8e',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '90',
   'date': 'Mon, 24 Jan 2022 16:34:20 GMT'},
  'RetryAttempts': 0}}

In [101]:
execution = pipeline.start()

In [102]:
execution.wait()

In [104]:
import json
import boto3

s3_client = boto3.client('s3')

uri = evaluation_s3_uri

obj = s3_client.get_object(Bucket=uri.split("/")[2], Key="/".join(uri.split("/")[3:]))

report = json.loads(obj['Body'].read())
#df = pd.read_csv(io.BytesIO(obj['Body'].read()))


#with open(evaluation_output_uris["evaluation"]+ "/evaluation.json") as jf:
#    report = json.load(jf)
    
print(report)

{'regression_metrics': {'accuracy': 0.8363423076923077, 'precision': 0.6699066830139103, 'recall': 0.44946899841182447, 'f1_score': 0.5366143977939883}}


### The model has a high enough accuracy and we like the other stats (not really but let's say so), so we can go and manually approve the model to join the registry.

We can now develop a deployment script that will get our model and deploy it.

P.s. If we don't like the manual approval of models we can change `model_approval_status` to `'Approved'`.

In [139]:
!git pull origin main

remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 7 (delta 6), reused 7 (delta 6), pack-reused 0
Unpacking objects: 100% (7/7), 579 bytes | 193.00 KiB/s, done.
From https://github.com/gmguarino/sagemaker-pipeline-rain-australia-build
 * branch            main       -> FETCH_HEAD
   241a27f..a41dda7  main       -> origin/main
Updating 241a27f..a41dda7
Fast-forward
 deploy_utils.py              | 8 ++------
 pipelines/rain/inference.py  | 2 +-
 pipelines/rain/preprocess.py | 1 +
 3 files changed, 4 insertions(+), 7 deletions(-)


In [107]:
!ls

codebuild-buildspec.yml  pipelines			  README.md  tests
deploy_utils.py		 rain-au-notebook-dev.ipynb	  setup.cfg
LICENSE			 rain-au-notebook-pipeline.ipynb  setup.py


We can get some helper code from `deploy_utils.py`

In [109]:
from deploy_utils import get_latest_approved_package

Let's print out the description for our model

In [113]:
sm_client = boto3.client("sagemaker")

# model_package_group_name was defined at the beginning of the pipeline definition 
package = get_latest_approved_package(model_package_group_name) 

model_pak_arn = package["ModelPackageArn"]

model_description = sm_client.describe_model_package(ModelPackageName=model_pak_arn)

model_description

{'ModelPackageGroupName': 'RainAuModel',
 'ModelPackageVersion': 1,
 'ModelPackageArn': 'arn:aws:sagemaker:eu-west-3:543553163241:model-package/rainaumodel/1',
 'CreationTime': datetime.datetime(2022, 1, 24, 16, 50, 29, 451000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '659782779980.dkr.ecr.eu-west-3.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3',
    'ImageDigest': 'sha256:fc21b1b187c1980fa792c9bd648c34d60e8622ad0963c140a8eb60ceef2fc549',
    'ModelDataUrl': 's3://sagemaker-eu-west-3-543553163241/pipelines-83o705b9nxst-sklearnRepackModel-gO4lDripba/output/model.tar.gz',
    'Environment': {'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
     'SAGEMAKER_PROGRAM': 'preprocess.py',
     'SAGEMAKER_REGION': 'eu-west-3',
     'SAGEMAKER_SUBMIT_DIRECTORY': 's3://sagemaker-eu-west-3-543553163241/sagemaker-scikit-learn-2022-01-24-15-41-47-116/sourcedir.tar.gz'}},
   {'Image': '763104351884.dkr.ecr.eu-west-3.amazonaws.com/pytorch-inference:1.8.0-cpu-py3',
    'ImageD

In [114]:
# We can now load it using the sagemaker ModelPackage class
from sagemaker import ModelPackage

import time

In [115]:
deploy_model = ModelPackage(
    role=role,
    model_package_arn=model_pak_arn,
    sagemaker_session=sagemaker_session
)

endpoint_name = "RainAu-test-deployment-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
# Deploy the model
deploy_model.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge", endpoint_name=endpoint_name)

--------------!

# Create a Predictor from an endpoint

In [122]:
from sagemaker.predictor import Predictor

predictor = Predictor(endpoint_name=endpoint_name)
# Can use this to test

In [117]:
#load test data
test_data_bucket = processing_step.arguments["ProcessingOutputConfig"]["Outputs"][2]["S3Output"]["S3Uri"]
test_data_bucket

's3://sagemaker-eu-west-3-543553163241/RainAuProcessing-e1284f24b646de14ac1aa45ecdee541d/output/test'

In [126]:
test_data = pd.read_csv(test_data_bucket + "/test.csv")
test_data.head()

,RainTomorrow,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,year,month_sin,month_cos,day_sin,day_cos
0,0,-0.196331,-0.265806,-0.764067,-0.203581,-0.622806,-0.503988,1.258262,-0.530619,1.328766,...,1.025756,-0.336969,-0.463168,-0.623142,-0.529795,0.878855,-1.244369,-0.682476,-1.317684,0.523473
1,0,0.365915,1.100882,0.505625,0.821481,-0.119472,-1.156687,-0.446007,1.298526,0.000105,...,0.149133,1.561680,0.901326,0.179704,1.887521,0.878855,1.211519,-0.682476,1.255292,-0.601030
2,0,0.857881,-1.711038,-1.427128,-0.155903,-0.119472,0.148710,0.406127,-2.054906,0.221549,...,0.149133,0.137693,-1.284966,-1.338404,-0.529795,0.090732,-0.725379,-1.198979,-1.261688,-0.601030
3,1,-0.266612,-1.051258,0.011856,-0.275097,0.069278,-0.286422,-0.872075,1.603383,-0.664226,...,-2.042425,1.561680,-0.122045,-0.083046,-0.529795,1.272917,-1.434333,0.023080,-0.803968,1.199371
4,0,0.787600,0.205466,0.181148,-0.108226,-0.119472,0.148710,1.045228,-0.378190,1.328766,...,-1.604113,1.087018,-0.060022,0.106718,1.887521,1.666978,0.692529,1.245139,-1.261688,-0.601030


In [134]:
batch_size = 10
y = test_data["RainTomorrow"].values
test_payload = test_data.drop(columns="RainTomorrow").iloc[:batch_size].values.tolist()

In [140]:
json_payload = {"data":test_payload} 

#There is an import error in inference.py FIX!
#p = predictor.predict(json.dumps(json_payload), initial_args={"ContentType": "application/json"})

In [141]:
!git add rain-au-notebook-dev.ipynb rain-au-notebook-pipeline.ipynb

In [146]:
!git commit -m "added notebooks"

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   rain-au-notebook-pipeline.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	pipelines/rain/_repack_model.py

no changes added to commit (use "git add" and/or "git commit -a")
